# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Анализ базы данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Начнем с анализа отдельных таблиц.

## Таблица books

In [2]:
query = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)

display(books.head())
books.info()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


## Таблица authors

In [3]:
query = '''
            SELECT * 
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)

display(authors.head())
authors.info()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


## Таблица publishers

In [4]:
query = '''
            SELECT * 
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)

display(publishers.head())
publishers.info()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


## Таблица ratings

In [5]:
query = '''
            SELECT * 
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)

display(ratings.head())
ratings.info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


## Таблица reviews

In [6]:
query = '''
            SELECT * 
            FROM ratings
        '''
reviews = pd.io.sql.read_sql(query, con = engine)

display(reviews.head())
reviews.info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


# Задания

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
            SELECT COUNT(book_id) AS count_of_books
            FROM books 
            WHERE publication_date > '01-01-2000'
        '''

pd.io.sql.read_sql(query, con = engine)

,count_of_books
0,819


В таблице `books` содержится 819 книг, вышедших после 1 января 2000 года

## Для каждой книги посчитайте количество обзоров и среднюю оценку


In [8]:
query = '''
            WITH book_rating AS
            (SELECT book_id,
            AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY book_id),

            book_review AS
            (SELECT book_id,
            COUNT(*) AS review_count
            FROM reviews
            GROUP BY book_id)
            
            
            SELECT b.title, b.book_id, avg_rating, review_count
            FROM books AS b
            LEFT JOIN book_rating AS b_rat ON b.book_id = b_rat.book_id
            LEFT JOIN book_review AS b_rev ON b.book_id = b_rev.book_id
            ORDER BY avg_rating DESC
            
        '''

pd.io.sql.read_sql(query, con = engine)

,title,book_id,avg_rating,review_count
0,A Dirty Job (Grim Reaper #1),17,5.00,4.0
1,Tai-Pan (Asian Saga #2),610,5.00,2.0
2,School's Out—Forever (Maximum Ride #2),553,5.00,3.0
3,Piercing the Darkness (Darkness #2),513,5.00,2.0
4,The War of Art: Break Through the Blocks & Win...,902,5.00,2.0
...,...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,915,2.25,3.0
996,His Excellency: George Washington,316,2.00,2.0
997,Junky,371,2.00,2.0
998,Drowning Ruth,202,2.00,3.0


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query = '''
            SELECT publisher, COUNT(book_id)
            FROM books AS b
            LEFT JOIN publishers AS p ON p.publisher_id = b.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY count DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
query = '''
            WITH rating AS
            (SELECT book_id, AVG(rating) ratings
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(*) >= 50)
            
            SELECT a.author, AVG(ratings)
            FROM rating
            JOIN (SELECT book_id, author_id FROM books) AS b ON rating.book_id=b.book_id
            JOIN (SELECT author_id, author FROM authors) AS a ON b.author_id=a.author_id
            GROUP BY a.author
            ORDER BY avg(ratings) DESC
            LIMIT 1 
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''
            SELECT AVG(reviews_quantity)
            FROM
            (SELECT username, COUNT(review_id) reviews_quantity
            FROM reviews
            WHERE username IN(
            SELECT username
            FROM
            (SELECT username, COUNT(rating_id) ratings_quantity
            FROM ratings
            GROUP BY username) AS t
            WHERE ratings_quantity > 50)
            GROUP BY username) AS tt
        '''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


# Заключение

В рамках проекта мы узнали:
- База данных содержит информацию о 1000 книг
- 819 книг было издано после 1 января 2020 года
- Больше всего книг длиннее 50 страниц выпустило издательство Penguin Books -	42 книги
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (Mary GrandPré - иллюстратор книг о Гарри Поттере)
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24